In [2]:
import re
import pdfplumber
import docx2txt

In [4]:
def read_pdf(file) -> str:
    text = ""
    with pdfplumber.open(file) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
        return text


def read_docx(file) -> str:
    return docx2txt.process(file)


def extract_info(text: str) -> dict:
    info = {
        "name": "",
        "email": "",
        "phone": "",
        "education": "",
        "experience": "",
    }

    email_match = re.search(r"[\w\.-]+@[\w\.-]+", text)
    phone_match = re.search(r"(\+62|08)[0-9\s\-]{8,}", text)

    if email_match:
        info["email"] = email_match.group(0)

    if phone_match:
        info["phone"] = phone_match.group(0)

    lines = text.strip().split("\n")

    for line in lines:
        if line.strip():
            info["name"] = " ".join(line.strip().split())[:3]
            break

    if "Universitas" in text or "SMA" in text:
        info["education"] = "Terdeksi pendidikan"

    if "PT" in text or "freelance" in text.lower():
        info["experience"] = "Terdeksi pengalaman kerja"

    return info

In [18]:
import ipywidgets as widget
from IPython.display import display, HTML
import pdfplumber
from PIL import Image

# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

upload = widget.FileUpload(
    accept='.pdf',
    multiple=False
)
display(upload)

FileUpload(value=(), accept='.pdf', description='Upload')

In [20]:
def read_pdf_from_upload(uploaded_file):
    file_content = uploaded_file['content']
    with pdfplumber.open(io.BytesIO(file_content)) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def full_text(text):
    display(HTML(f"""
        <div style='white-space: pre-wrap; font-family: monospace; 
                    max-height: 600px; overflow-y: auto; 
                    border: 1px solid #ddd; padding: 10px;'>
            {text}
        </div>
    """))

if upload.value:
    uploaded_file = upload.value[0]  
    pdf_text = read_pdf_from_upload(uploaded_file)
    full_text(pdf_text)
else:
    print("Silakan upload file PDF terlebih dahulu.")

In [ ]:
from pprint import pprint

def extractCVData(pdf_text):
    lines = [line.strip() for line in pdf_text.split('\n') if line.strip()]

    result = {
        "name" : lines[0],
        "position" : lines[1],
        "contact" : {},
        "summary" : "",
        "experience" : [],
        "education" : [],
        "skills" : [],       
    }

    phone_regex = r'08\d{2}[\s-]?\d{4}[\s-]?\d{4}'
    email_regex = r'[\w\.-]+@[\w\.-]+'
    section = None
    buffer = []

    
    for line in lines:
        # Kontak
        if re.search(phone_regex, line):
            result['contact']['phone'] = re.search(phone_regex, line).group()
        if re.search(email_regex, line):
            result['contact']['email'] = re.search(email_regex, line).group()
        if 'Jl.' in line or 'Perum' in line:
            result['contact']['address'] = line
        
        # Deteksi section
        if 'Pengalaman Kerja' in line:
            section = 'experience'
            continue
        elif 'Pendidikan' in line:
            section = 'education'
            continue
        elif 'Skill' in line:
            section = 'skills'
            continue

        # Simpan berdasarkan section
        if section == 'experience':
            if re.search(r'\d{4}', line):
                buffer.append(line)
            elif buffer:
                buffer[-1] += ' ' + line
        elif section == 'education':
            result['education'].append(line)
        elif section == 'skills':
            result['skills'].append(line)
        elif section is None and not result['summary']:
            result['summary'] += ' ' + line

    # Format pengalaman kerja (opsional lebih lanjut bisa parsing tanggal)
    result['experience'] = buffer

    return result

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3898891009.py, line 11)